In [12]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt

In [13]:
batch_size = 1024
block_size = 128

vocab_size = 256
num_embed = 192
num_head = 6
num_layer = 6

### Data loader

In [14]:
train = np.memmap("train.bin", dtype=np.uint8, mode="r")

In [15]:
def get_batch():
    indices = torch.randint(len(train) - block_size, (batch_size,))
    X = torch.stack([torch.from_numpy((train[i:i+block_size]).astype(np.int64)) for i in indices]).to('cuda')
    Y = torch.stack([torch.from_numpy((train[i+1:i+1+block_size]).astype(np.int64)) for i in indices]).to('cuda')
    return X, Y

### Model definition

In [16]:
class MlpBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(num_embed, 4 * num_embed)
        self.fc2 = nn.Linear(4 * num_embed, num_embed)
        self.gelu = nn.GELU()

    def forward(self, x):
        return self.fc2(self.gelu(self.fc1(x)))

class DecoderBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.norm1 = nn.LayerNorm(num_embed)
        self.attn = nn.MultiheadAttention(num_embed, num_head, batch_first=True)
        self.norm2 = nn.LayerNorm(num_embed)
        self.mlp = MlpBlock()
        self.register_buffer("attn_mask", nn.Transformer.generate_square_subsequent_mask(block_size).to('cuda'))

    def forward(self, x):
        x_length = x.shape[1]
        
        normalized_x = self.norm1(x)
        x = x + self.attn(normalized_x, normalized_x, normalized_x, attn_mask=self.attn_mask[:x_length, :x_length])[0]
        x = x + self.mlp(self.norm2(x))
        return x

class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, num_embed)
        self.position_embedding = nn.Embedding(block_size, num_embed)
        self.blocks = nn.Sequential(*[DecoderBlock() for _ in range(num_layer)])
        self.norm = nn.LayerNorm(num_embed)
        self.head = nn.Linear(num_embed, vocab_size, bias=False)

    def forward(self, x):
        x_length = x.shape[1]
        positions = torch.arange(0, x_length, dtype=torch.long)[None,:].to('cuda')
        token_x = self.token_embedding(x)
        position_x = self.position_embedding(positions)
        x = token_x + position_x
        x = self.blocks(x)
        x = self.norm(x)
        x = self.head(x)
        return x

In [17]:
x, y = get_batch()
model = GPT().to('cuda')
output = model(x)
bytes(torch.argmax(output, dim=-1)[0])

RuntimeError: handle_0 INTERNAL ASSERT FAILED at "/opt/conda/conda-bld/pytorch_1695392020201/work/c10/cuda/driver_api.cpp":15, please report a bug to PyTorch. 

In [ ]:
def calc_loss(logits, target):
    return F.cross_entropy(logits.view(-1, logits.size(-1)), target.view(-1))

### Implementing learning rate finder

In [ ]:
model = GPT().to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=1.3)

In [ ]:
min_loss = 1e9
lrs = []
losses = []
while True:
    optimizer.zero_grad()
    x, y = get_batch()
    loss = calc_loss(model(x), y)
    current_loss = float(loss)
    min_loss = min(min_loss, current_loss)
    if min_loss*3 < current_loss:
        break
    lrs.append(scheduler.get_last_lr()[0])
    losses.append(current_loss)
    if len(lrs) % 20 == 0:
        print("loss = %f lr = %f" % (losses[-1], lrs[-1]))
    loss.backward()
    optimizer.step()
    scheduler.step()

In [ ]:
plt.plot(lrs, losses)
plt.xlabel("learning rate")
plt.ylabel("loss")
plt.xscale('log')
plt.show()

### Training loop

In [7]:
model = GPT().to('cuda')
total_steps = 15000
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-2, total_steps=total_steps)

In [8]:
for i in (pbar := tqdm(range(total_steps))):
    optimizer.zero_grad()
    x, y = get_batch()
    loss = calc_loss(model(x), y)
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    pbar.set_description("lr = %f loss = %f" % (scheduler.get_last_lr()[0], loss))
    
    if i > 0 and i % 2000 == 0:
        torch.save({
            'step': i,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': loss,
            }, "ckpt-%d" % i)

lr = 0.000000 loss = 1.548402: 100%|████████████████████████████████████████████| 15000/15000 [1:07:28<00:00,  3.70it/s]


In [9]:
torch.save({
    'step': total_steps,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'loss': loss,
    }, "ckpt-%d" % total_steps)

In [14]:
# x, y = get_batch()
output = model(torch.tensor([c for c in "articl".encode("utf-8")])[None,:].to('cuda'))
bytes(torch.argmax(output, dim=-1)[0])

b'nt sle'

In [17]:
torch.argmax(output, dim=-1)[0]

tensor([104, 101,  32,  97, 101, 114, 101, 108,  32, 115, 116,  32, 111,  97,
        115,  32,  97, 111, 114, 110,  32, 105, 110,  32, 116],
       device='cuda:0')

In [49]:
cur = [c for c in "I'm not a cat, ".encode("utf-8")]
for i in range(30):
    output = model(torch.tensor(cur)[None,:].to('cuda'))
    cur.append(int(torch.argmax(output, dim=-1)[0, -1]))
print(bytes(cur))

b"I'm not a cat, and the second of the second o"


In [72]:
cur = [c for c in "Wonderful".encode("utf-8")]
for i in range(128 - len(cur)):
    output = model(torch.tensor(cur)[None,:].to('cuda'))
    probs = F.softmax(output[0, -1, :], dim=0)
    cur.append(int(torch.multinomial(probs, num_samples=1)))
print(bytes(cur).decode("utf-8"))

Wonderful Records on the coasts children in 1398. In the titled in 2010, depened White received the following cineman was April 
